In [4]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
import scipy as sp

def find_if_close(cnt1,cnt2,const):
    M1 = cv2.moments(cnt1)
    M2 = cv2.moments(cnt2)
    
    if(M1["m00"]!=0 and M2["m00"]!=0):
        cX1 = int((M1["m10"] / M1["m00"]))
        cY1 = int((M1["m01"] / M1["m00"]))
        cX2 = int((M2["m10"] / M2["m00"]))
        cY2 = int((M2["m01"] / M2["m00"]))
        
        dist = np.sqrt((cX1-cX2)*(cX1-cX2)+(cY1-cY2)*(cY1-cY2)) 
        if dist < const :
            return True,(cX1,cY1),(cX2,cY2)
    return False,(0,0),(0,0)        
def saveMatInDir(mat,dir,name):
    cv2.imwrite(dir+name, mat)
#cap = cv2.VideoCapture('C:\\Users\\egor\\Documents\\20161208_074126.mp4')
#cap = cv2.VideoCapture('20160519_081947.mp4')

#cap = cv2.VideoCapture("D:\\vidsnew\\vid_last.mp4")
#cap.set(cv2.CAP_PROP_POS_MSEC,161000)
#cap.set(cv2.CAP_PROP_POS_MSEC,1510000)
fgbg = cv2.createBackgroundSubtractorMOG2(100,5,False)
perFrame=True
orb = cv2.ORB_create()
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
iii = 30
frame = cv2.imread('C:\\Users\\egor\\Documents\\github\\diploma\\ball_try\\L_try2\\'+str(iii).zfill(9)+".png",0)
fgmask = fgbg.apply(frame)
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3))
kernel1 = cv2.getStructuringElement(cv2.MORPH_CROSS,(5,5))
oldUnified=[]
unified = []
linearr=[]
counter=0
while(1):
    iii+=1
    #ret, frame = cap.read()
    frame = cv2.imread('C:\\Users\\egor\\Documents\\github\\diploma\\ball_try\\L_try2\\'+str(iii).zfill(9)+".png",0)
    baseframe=frame.copy()
    fgmask = fgbg.apply(frame)
    fgmask = cv2.morphologyEx(fgmask, cv2.MORPH_OPEN, kernel)
    
    black = np.zeros_like(fgmask)
    res=np.zeros_like(frame)
    #res = cv2.bitwise_and(frame,frame,mask = fgmask)
   
    #fgmask = cv2.dilate(fgmask, kernel1, iterations=5)

    ret,thresh = cv2.threshold(fgmask,127,255,cv2.THRESH_BINARY)
    im2, contours, hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    if(len(contours)>100):
        continue
        
    #######
    LENGTH = len(contours)
    status = np.zeros((LENGTH,1))

    for i,cnt1 in enumerate(contours):
        x = i    
        if i != LENGTH-1:
            for j,cnt2 in enumerate(contours[i+1:]):
                x = x+1
                dist,bbbb,bbbbbb = find_if_close(cnt1,cnt2,50)
                if dist == True:
                    val = min(status[i],status[x])
                    status[x] = status[i] = val
                else:
                    if status[x]==status[i]:
                        status[x] = i+1
    sizeold=len(linearr)
    rectArr=[]
    if(len(status>0)):
        oldUnified=list(unified)
        unified=[]
        maximum = int(status.max())+1
        for i in range(maximum):
            pos = np.where(status==i)[0]
            if pos.size != 0:
                cont = np.vstack(contours[i] for i in pos)
                x,y,w,h = cv2.boundingRect(cont)
                #cv2.rectangle(frame,(x,y),(x+w,y+h),(150,60,60),2)
                rectArr.append((x,y,w,h))
    if(len(rectArr)>50):
        continue
        
    cv2.imshow('asdasdf',fgmask)
    if(perFrame==True):
        ind=0
        while ind < len(rectArr):            
            x,y,w,h=rectArr[ind]
            deltaX=int((80-w)/2)
            deltaY=int((80-h)/2)
            if(y-deltaY<0 or x-deltaX<0 or x-deltaX+80>len(baseframe[0]) or y-deltaY+80>len(baseframe)):
                ind+=1
                continue
            frame=baseframe.copy()            
            cv2.rectangle(frame,(x-deltaX,y-deltaY),(x-deltaX+80,y-deltaY+80),(0,0,255),2)
            roi=fgmask[y-deltaY:y-deltaY+80,x-deltaX:x-deltaX+80]
            cv2.imshow('asdasdasff',frame)
            key = cv2.waitKey(0)
            if(key==ord('a')):
                saveMatInDir(roi,'C:\\Users\\egor\\Documents\\notBallDir\\',str(iii)+str(ind)+'.jpg')
            if(key==ord('b')):
                saveMatInDir(roi,'C:\\Users\\egor\\Documents\\ballDir\\',str(iii)+'_'+str(ind)+'.jpg')
            if(key==ord('i')):
                frame=baseframe.copy()            
                cv2.rectangle(frame,(x-deltaX,y-deltaY),(x-deltaX+80,y-deltaY+80),(0,0,255),2)
                roi=fgmask[y-deltaY:y-deltaY+80,x-deltaX:x-deltaX+80]
                frame[y-deltaY:y-deltaY+80,x-deltaX:x-deltaX+80]=roi
                cv2.imshow('asdasdasff',frame)
                first=False
                key = cv2.waitKey(0)
                while(key!=8):
                    frame=baseframe.copy()            
                    cv2.rectangle(frame,(x-deltaX,y-deltaY),(x-deltaX+80,y-deltaY+80),(0,0,255),2)
                    roi=fgmask[y-deltaY:y-deltaY+80,x-deltaX:x-deltaX+80]
                    if first:
                        frame[y-deltaY:y-deltaY+80,x-deltaX:x-deltaX+80]=roi
                    first=not first
                    cv2.imshow('asdasdasff',frame)
                    key = cv2.waitKey(0)
                ind-=1
            if(key==27):
                break
            if(key==32):
                perFrame=False
            ind+=1
            #cv2.rectangle(frame,(x,y),(x+w,y+h),(150,60,60),2)
    
    #cv2.drawContours(black,unified,-1, (255,255,255), -1)
    #cv2.drawContours(frame,unified,-1, (0,255,0), 2)
   
    #cv2.imshow('asdasdff',frame)
    
    if(len(rectArr)>0 and perFrame==True):
        continue
    delay = 1
    k = cv2.waitKey(delay)
    if(k!=-1):
        print (k)
    if k == 27:
        break
    if k == 112:
        cv2.imwrite('pic{:>05}.jpg'.format(iii), res)
    if k == 32 or perFrame==True:
        k = cv2.waitKey(0)
        if(k==32):
            perFrame=False
            continue
        if k == 112:
            cv2.imwrite('pic{:>05}.jpg'.format(iii), res)
            cv2.imwrite('picr{:>05}.jpg'.format(iii), res1)
        if(k == 2555904): 
            perFrame=True
            continue
    
        
    

#cap.release()
cv2.destroyAllWindows()

27


In [3]:
import numpy as np
import cv2
from skimage import morphology
from matplotlib import pyplot as plt
import scipy as sp
import yaml

def opencv_matrix(loader, node):
    mapping = loader.construct_mapping(node, deep=True)
    mat = np.array(mapping["data"])
    mat.resize(mapping["rows"], mapping["cols"])
    return mat
yaml.add_constructor(u"tag:yaml.org,2002:opencv-matrix", opencv_matrix)

with open("C:\\Users\\egor\\Documents\\Visual Studio 2013\\Projects\\calib\\calib\\intrinsics.yml", 'r') as fin:
    try:
        result = yaml.load(fin.read())
    except yaml.YAMLError as exc:
        print(exc)

cameraMatrix0=result["M1"]
cameraMatrix1=result["M2"]
distCoeffs0=result["D1"]
distCoeffs1=result["D2"]
with open("C:\\Users\\egor\\Documents\\Visual Studio 2013\\Projects\\calib\\calib\\extrinsics.yml", 'r') as fin:
    try:
        result = yaml.load(fin.read())
    except yaml.YAMLError as exc:
        print(exc)
R1=result["R1"]
R2=result["R2"]
P1=result["P1"]
P2=result["P2"]
imageSizeCropped=result["imageSizeCropped"]
Q=result["Q"]
w = imageSizeCropped[0]
h= imageSizeCropped[1]
    
def opencv_matrix(loader, node):
    mapping = loader.construct_mapping(node, deep=True)
    mat = np.array(mapping["data"])
    mat.resize(mapping["rows"], mapping["cols"])
    return mat
def skeletonize(matIn):
    mat = morphology.skeletonize(matIn > 0)
    black = np.zeros_like(matIn)
    t=np.array([[255 if el==True else 0 for el in row] for row in mat])
    return t
def getLeftRight_rectificated(imgLIn,imgRIn):
    
    
    rm00,rm01=cv2.initUndistortRectifyMap(cameraMatrix0, distCoeffs0, R1, P1, tuple(imageSizeCropped), cv2.CV_16SC2)
    rm10,rm11=cv2.initUndistortRectifyMap(cameraMatrix1, distCoeffs1, R2, P2, tuple(imageSizeCropped), cv2.CV_16SC2)
    imgL=cv2.remap(imgLIn.copy(),rm00,rm01,cv2.INTER_LINEAR)
    imgR=cv2.remap(imgRIn.copy(),rm10,rm11,cv2.INTER_LINEAR)
    return imgL,imgR

def find_if_close(cnt1,cnt2,const):
    M1 = cv2.moments(cnt1)
    M2 = cv2.moments(cnt2)
    
    if(M1["m00"]!=0 and M2["m00"]!=0):
        cX1 = int((M1["m10"] / M1["m00"]))
        cY1 = int((M1["m01"] / M1["m00"]))
        cX2 = int((M2["m10"] / M2["m00"]))
        cY2 = int((M2["m01"] / M2["m00"]))
        
        dist = np.sqrt((cX1-cX2)*(cX1-cX2)+(cY1-cY2)*(cY1-cY2)) 
        if dist < const :
            return True,(cX1,cY1),(cX2,cY2)
    return False,(0,0),(0,0)        
def saveMatInDir(mat,dir,name):
    cv2.imwrite(dir+name, mat)
def getFgMask_withBall(fgmask,baseframe,rectArr):
    ind=0
    black = np.zeros_like(baseframe)
    
    found=False
    while ind < len(rectArr):            
        x,y,w,h=rectArr[ind]
        deltaX=int((80-w)/2)
        deltaY=int((80-h)/2)
        if(y-deltaY<0 or x-deltaX<0 or x-deltaX+80>len(baseframe[0]) or y-deltaY+80>len(baseframe)):
            ind+=1
            continue
        frame=baseframe.copy()            
        cv2.rectangle(frame,(x-deltaX,y-deltaY),(x-deltaX+80,y-deltaY+80),(0,0,255),2)
        roi=fgmask[y-deltaY:y-deltaY+80,x-deltaX:x-deltaX+80]
        cv2.imshow('MARKmode',frame)
        key = cv2.waitKey(0)
        if(key==ord('a')):
            ind+=1
            continue
        if(key==ord('b')):
            black[y-deltaY:y-deltaY+80,x-deltaX:x-deltaX+80]=roi
            found=True
            break
        ind+=1
    return found,black
def getRectArr(contours):
    LENGTH = len(contours)
    status = np.zeros((LENGTH,1))
    
    for i,cnt1 in enumerate(contours):
        x = i    
        if i != LENGTH-1:
            for j,cnt2 in enumerate(contours[i+1:]):
                x = x+1
                dist,bbbb,bbbbbb = find_if_close(cnt1,cnt2,50)
                if dist == True:
                    val = min(status[i],status[x])
                    status[x] = status[i] = val
                else:
                    if status[x]==status[i]:
                        status[x] = i+1
    rectArr=[]
    if(len(status>0)):
        unified=[]
        maximum = int(status.max())+1
        for i in range(maximum):
            pos = np.where(status==i)[0]
            if pos.size != 0:
                cont = np.vstack(contours[i] for i in pos)
                x,y,w,h = cv2.boundingRect(cont)
                #cv2.rectangle(frame,(x,y),(x+w,y+h),(150,60,60),2)
                rectArr.append((x,y,w,h))
    return rectArr
#cap = cv2.VideoCapture('C:\\Users\\egor\\Documents\\20161208_074126.mp4')
#cap = cv2.VideoCapture('20160519_081947.mp4')

#cap = cv2.VideoCapture("D:\\vidsnew\\vid_last.mp4")
#cap.set(cv2.CAP_PROP_POS_MSEC,161000)
#cap.set(cv2.CAP_PROP_POS_MSEC,1510000)
fgbgLeft = cv2.createBackgroundSubtractorMOG2(100,5,False)
fgbgRight = cv2.createBackgroundSubtractorMOG2(100,5,False)
perFrame=True
orb = cv2.ORB_create()
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
iii = 292
frameLeftDirty = cv2.imread('C:\\Users\\egor\\Documents\\github\\diploma\\ball_try\\L_try2\\'+str(iii).zfill(9)+".png",0)
frameRightDirty = cv2.imread('C:\\Users\\egor\\Documents\\github\\diploma\\ball_try\\R_try2\\'+str(iii).zfill(9)+".png",0)
frameLeft,frameRight=getLeftRight_rectificated(frameLeftDirty,frameRightDirty)    

fgmaskLeft = fgbgLeft.apply(frameLeft)
fgmaskRight = fgbgRight.apply(frameRight)

kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3))

while(1):
    iii+=1
    #ret, frame = cap.read()
    frameLeftDirty = cv2.imread('C:\\Users\\egor\\Documents\\github\\diploma\\ball_try\\L_try2\\'+str(iii).zfill(9)+".png",0)
    frameRightDirty = cv2.imread('C:\\Users\\egor\\Documents\\github\\diploma\\ball_try\\R_try2\\'+str(iii).zfill(9)+".png",0)
    frameLeft,frameRight=getLeftRight_rectificated(frameLeftDirty,frameRightDirty)    
    baseframeLeft=frameLeft.copy()
    baseframeRight=frameRight.copy()
    
    fgmaskLeft = fgbgLeft.apply(frameLeft)
    fgmaskLeft = cv2.morphologyEx(fgmaskLeft, cv2.MORPH_OPEN, kernel)
    
    fgmaskRight = fgbgRight.apply(frameRight)
    fgmaskRight = cv2.morphologyEx(fgmaskRight, cv2.MORPH_OPEN, kernel)
    
    
    blackLeft = np.zeros_like(fgmaskLeft)
    blackRight = np.zeros_like(fgmaskRight)
    
    ret,threshLeft = cv2.threshold(fgmaskLeft,127,255,cv2.THRESH_BINARY)
    im2, contoursLeft, hierarchy = cv2.findContours(threshLeft,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    
    ret,threshRight = cv2.threshold(fgmaskRight,127,255,cv2.THRESH_BINARY)
    im2, contoursRight, hierarchy = cv2.findContours(threshRight,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    
    print(len(contoursLeft))
    
    if(len(contoursLeft)>100):
        continue
    
    #######
    rectArrLeft=getRectArr(contoursLeft)
    rectArrRight=getRectArr(contoursRight) 
    
    if(len(rectArrLeft)>50):
        continue  
    
    
    if(perFrame==True): 
        found,leftBallBlack=getFgMask_withBall(fgmaskLeft,baseframeLeft,rectArrLeft)
        if(found):
            leftBallTraj=skeletonize(leftBallBlack)
            found,rightBallBlack=getFgMask_withBall(fgmaskRight,baseframeRight,rectArrRight)
            if(found):
                rightBallTraj=skeletonize(rightBallBlack)
                cv2.imwrite('C:\\Users\\egor\\Documents\\github\\diploma\\ball_center\\L\\{:>09}.jpg'.format(iii), leftBallTraj)
                cv2.imwrite('C:\\Users\\egor\\Documents\\github\\diploma\\ball_center\\R\\{:>09}.jpg'.format(iii), rightBallTraj)
                k = cv2.waitKey(0)
                if(k==27):
                    perFrame=False
                
        
        #cv2.rectangle(frame,(x,y),(x+w,y+h),(150,60,60),2)
    
    #cv2.drawContours(black,unified,-1, (255,255,255), -1)
    #cv2.drawContours(frame,unified,-1, (0,255,0), 2)
   
    #cv2.imshow('asdasdff',frame)    
    if(perFrame==False):
        break
    delay = 1
    k = cv2.waitKey(delay)
    
    if(k!=-1):
        print (k)
    if k == 27:
        break
    else:
        continue
    if k == 112:
        cv2.imwrite('pic{:>05}.jpg'.format(iii), res)
    if k == 32 or perFrame==True:
        k = cv2.waitKey(0)
        if(k==32):
            perFrame=False
            continue
        if k == 112:
            cv2.imwrite('pic{:>05}.jpg'.format(iii), res)
            cv2.imwrite('picr{:>05}.jpg'.format(iii), res1)
        if(k == 2555904): 
            perFrame=True
            continue
    
        
    

#cap.release()
cv2.destroyAllWindows()

63
50
53
43
37
51
56
48
38
30
38
30
31
40
55
44
34
32
27
31
24
13
25
22
23
24
14
16
12
10
15
10
12
15
8
9
8
9
10
10
6
4
7
5
27


In [10]:
import cv2
img = cv2.imread('C:\\Users\\egor\\Documents\\notBallDir\\332.jpg') # load a dummy image
while(1):
    cv2.imshow('img',img)
    k = cv2.waitKey(33)
    if k==27:    # Esc key to stop
        break
    elif k==-1:  # normally -1 returned,so don't print it
        continue
    else:
        print (k) # else print its value

97
97
8
